In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict
from spotipy.oauth2 import SpotifyOAuth


client_id="ba503ee919b241a19afb3a14415d3095"
client_secret = "41ade86e21e34517a44c1daa6102f723"

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id, client_secret))

Get playlist data and save it to json file to be used later

In [3]:
import json
import time
#from spotify_client_credentials import *

client_credentials_manager = SpotifyClientCredentials(client_id,client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


def get_track_ids(playlist_id):
    music_id_list = []
    playlist = sp.playlist(playlist_id)
    for item in playlist['tracks']['items']:
        music_track = item['track']
        music_id_list.append(music_track['id'])
    return music_id_list

def get_track_data(track_id):
    meta = sp.track(track_id)
    track_details = {"name":meta['name'],"album":meta['album']['name'],"artist":meta['album']['artists'][0]['name'],
                     "release_date":meta['album']['release_date'], "duration_in_mins":round((meta['duration_ms']*.001)/60,2)}
    return track_details

playlist_id = '1JgJbkOC4RpbLIeDNO8xGB'
playlist_name = "''"+sp.playlist(playlist_id)['name']+"''"
track_ids = get_track_ids(playlist_id)

tracks = []

for i in range(len(track_ids)):
    time.sleep(.5)
    track = get_track_data(track_ids[i])
    tracks.append(track)

with open('spotify_playlsit_data.json','w') as outfile:
    json.dump(tracks,outfile,indent=4)

In [7]:
import requests
from bs4 import BeautifulSoup

In [16]:
f = open('spotify_playlsit_data.json')

playlist_data = json.load(f)
names = []
years = []
count = 0
reccomender_str = ""
dict_list = []
test = False
for i in range(len(playlist_data)):
    name = playlist_data[i]['name'].split(' ')
    hyphen_name = '' 
    for j in range(len(name)):
        if(name[j] == '-'):
            break
        for n in range(len(name[j])):
            if (name[j][n] == "'" or name[j][n] == "’"):#if there is an apostrophe
                hyphen_name += '%27'
            elif(name[j][n] == "&"):
                hyphen_name += '%26'
            else:
                hyphen_name += name[j][n]
                
        hyphen_name += '-'
    hyphen_name = hyphen_name[:-1]
        
    artist = playlist_data[i]['artist'].split(' ')
    hyphen_artist = ''
    
    for j in range(len(artist)):
        if(artist[j] == '-'):
            continue
        for n in range(len(artist[j])):
            if (artist[j][n] == "'" or artist[j][n] == "’"):#if there is an apostrophe
                hyphen_artist += '%27'
            elif(artist[j][n] == "&"):
                hyphen_artist += '%26'
            else:
                hyphen_artist += artist[j][n]
                
        hyphen_artist += '-'
    hyphen_artist = hyphen_artist[:-1]
    url = ''
    url = 'https://www.whosampled.com/'+str(hyphen_artist)+'/'+str(hyphen_name)
    print(url)
    
    samples_URL = url+'/samples/'
    page = requests.get(samples_URL)
    soup = BeautifulSoup(page.content, "html.parser")

    data = {}
    main = soup.find()
    sample_box = soup.find('div',class_="list bordered-list")
    print(sample_box)
    samples = sample_box.find_all('div', class_='listEntry sampleEntry')

    for sample in samples:

        track_details = sample.find('div', class_='trackDetails')
        details_inner = track_details.find('div', class_='details-inner')
        title = details_inner.find('a', class_='title').text
        track_artist = details_inner.find('span', class_='trackArtist').text
        artist = track_artist.find('a').text
        print(title,track_artist)

    year = int(playlist_data[i]['release_date'].split('-')[0])
    names.append(name)
    years.append(year)
    song_dict = dict(name=names[i],year=years[i])
    dict_list.append(song_dict)
    #print(song_dict)
f.close()

https://www.whosampled.com/The-Avalanches/Because-I%27m-Me
None


AttributeError: 'NoneType' object has no attribute 'find_all'